#TXT35系統,feature_v4,ver0.0.8から移植

In [12]:
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import xgboost as xgb
from matplotlib import pyplot as plt
from hyperopt import hp,fmin,tpe,STATUS_OK,Trials
import warnings
warnings.simplefilter('ignore')

In [2]:
data_dir = "F:/TARGET frontier/TXT35/"
data_name = "train"
tail = "fe1"
train_data = pd.read_csv(data_dir+data_name+tail+".csv",encoding="shift-jis",dtype={'レースID(新)':str})

In [3]:
pd.set_option('display.max_columns', 100)
train_data.head()

,レースID(新),場所,芝・ダ,距離,クラス名,馬場状態,出走頭数,騎手,枠番,性別,年齢,キャリア,斤量,馬体重,馬体重増減,間隔,休み明け〜戦目,種牡馬,父タイプ名,所属,調教師,入線順位,確定着順,着差,単勝オッズ,複勝オッズ下限,複勝オッズ上限,複勝配当,前走レースID(新),前走場所,前芝・ダ,前距離,前クラス名,前走馬場状態,前走出走頭数,替,前騎手,前走枠番,前走脚質,前走斤量,前走Ave-3F,前走上り3F,前走上り3F順,前走RPCI,前PCI,前好走,前走入線順位,前走確定着順,前走着差タイム,前走PCI3,前走単勝オッズ,場所距離芝ダ,前走場所距離芝ダ
0,201512270605081112,2,2,1200,2,3,16,427,6,2,3,13,56.0,492,-4.0,6.0,5.0,572,8,0,323,1,1,0.0,5.4,1.9,2.7,190.0,201511140505031003,13,2,1400.0,2,2,15.0,0,285,2.0,3,54.0,37.2,33.9,10.0,58.2,59.7,0,8.0,8.0,0.2,59.53,14.6,63,42
1,201512270605081111,2,2,1200,2,3,16,67,6,1,3,10,54.0,466,-12.0,3.0,3.0,452,8,0,322,2,2,0.0,2.8,1.3,1.6,130.0,201512060704020613,1,2,1200.0,1,3,18.0,1,67,7.0,3,54.0,34.5,34.6,3.0,46.9,49.7,0,1.0,1.0,0.0,48.80,2.2,63,15
2,201512270605081106,2,2,1200,2,3,16,209,3,2,3,9,56.0,478,6.0,5.0,2.0,607,5,1,210,3,3,0.2,6.2,1.8,2.5,180.0,201511210805051217,3,2,1200.0,2,3,18.0,0,137,8.0,2,56.0,34.5,34.4,9.0,48.0,50.3,0,7.0,7.0,0.4,50.13,28.6,63,188
3,201512270605081108,2,2,1200,2,3,16,338,4,1,5,17,55.0,484,-4.0,2.0,4.0,91,1,1,334,4,4,0.3,6.6,1.7,2.5,0.0,201512130605040911,2,2,1200.0,2,2,16.0,1,338,6.0,2,53.0,34.1,34.4,3.0,46.3,49.1,0,3.0,3.0,0.0,48.17,19.1,63,111
4,201512270605081116,2,2,1200,2,3,16,387,8,1,3,15,54.0,440,-6.0,2.0,4.0,325,4,1,162,5,5,0.3,33.9,6.3,9.7,0.0,201512130605040904,2,2,1200.0,2,2,16.0,0,213,2.0,5,53.0,33.6,36.4,15.0,46.3,42.3,1,13.0,13.0,1.5,48.17,8.6,63,111


In [4]:
train_data.columns

Index(['レースID(新)', '場所', '芝・ダ', '距離', 'クラス名', '馬場状態', '出走頭数', '騎手', '枠番', '性別',
       '年齢', 'キャリア', '斤量', '馬体重', '馬体重増減', '間隔', '休み明け〜戦目', '種牡馬', '父タイプ名',
       '所属', '調教師', '入線順位', '確定着順', '着差', '単勝オッズ', '複勝オッズ下限', '複勝オッズ上限',
       '複勝配当', '前走レースID(新)', '前走場所', '前芝・ダ', '前距離', '前クラス名', '前走馬場状態',
       '前走出走頭数', '替', '前騎手', '前走枠番', '前走脚質', '前走斤量', '前走Ave-3F', '前走上り3F',
       '前走上り3F順', '前走RPCI', '前PCI', '前好走', '前走入線順位', '前走確定着順', '前走着差タイム',
       '前走PCI3', '前走単勝オッズ', '場所距離芝ダ', '前走場所距離芝ダ'],
      dtype='object')

In [5]:
use_train_columns = [
    'クラス名', '馬場状態', '出走頭数', '騎手', '枠番', '性別',
       '年齢', 'キャリア', '斤量', '馬体重', '馬体重増減', '間隔', '休み明け〜戦目', '種牡馬', '父タイプ名',
       '所属', '調教師','単勝オッズ',
    '前走場所距離芝ダ','前クラス名', '前走馬場状態',
       '前走出走頭数', '替', '前騎手', '前走枠番', '前走脚質', '前走斤量', '前走Ave-3F', '前走上り3F',
       '前走上り3F順', '前走RPCI', '前PCI', '前好走', '前走入線順位','前走着差タイム',
       '前走PCI3', '前走単勝オッズ',
]

In [6]:
#test_data
test_data = pd.read_csv(data_dir+"predict"+tail+".csv",encoding="shift-jis",dtype={'レースID(新)':str})
test_x = test_data[use_train_columns]
test_y = test_data["入線順位"]

In [7]:
train_x = train_data[use_train_columns]
train_y = train_data["入線順位"]

In [8]:
#ベースライン
params = {
    'booster':'gbtree',
    'objective':'reg:squarederror',
    'eta':0.1,
    'gamma':0.0,
    'alpha':0.0,
    'lambda':1.0,
    'min_child_weight':1,
    'max_depth':6,
    'subsample':0.8,
    'colsample_bytree':0.8,
    'random_state':71,
}

In [9]:
#これがないとエラー履くので注意
train_y.fillna(10,inplace=True) #無ければ10位に
test_y.fillna(10,inplace=True)

In [10]:
def score(param_space):
    scores = []
    histories = []
    models = []
    
    for i,j in params.items():
        if i not in param_space:
            param_space[i] = j
    param_space['max_depth'] = int(param_space['max_depth'])
    print(param_space)    
    
    kf = KFold(n_splits=4,shuffle=False)
    for tr_idx,va_idx in kf.split(train_x):
        tr_x, va_x = train_x.iloc[tr_idx], train_x.iloc[va_idx]
        tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]
        dtrain = xgb.DMatrix(tr_x,label=tr_y)
        dvalid = xgb.DMatrix(va_x,label=va_y)
        watchlist = [(dtrain,"train"),(dvalid,"eval")]
        evals_result = {}
        model = xgb.train(
            param_space,
            dtrain,
            num_boost_round=10000,
            early_stopping_rounds=100,
            evals=watchlist,
            evals_result=evals_result,
            verbose_eval=100
        )
        models.append(model)
        histories.append(evals_result)
        va_pred = model.predict(dvalid)
        va_y.fillna(0.5,inplace=True)
        #print(va_pred[:10])
        #print(va_y[:10])
        score = np.sqrt(mean_squared_error(va_y,va_pred)) #rmse
        scores.append(score)
    rmse_score = np.mean(scores)
    history_opt.append((param_space,rmse_score))
    
    return {"loss": score, "status": STATUS_OK}

In [13]:
#パラメータチューニング
param_space = {

    'min_child_weight': hp.loguniform('min_child_weight', np.log(0.1), np.log(20)),

    'max_depth': hp.quniform('max_depth', 5, 9, 1),

    'subsample': hp.quniform('subsample', 0.6, 0.95, 0.05),

    'colsample_bytree': hp.quniform('colsample_bytree', 0.6, 0.95, 0.05),

    'gamma': hp.loguniform('gamma', np.log(1e-8), np.log(1.0)),

    # 余裕があればalpha, lambdaも調整する

    'alpha' : hp.loguniform('alpha', np.log(1e-8), np.log(1.0)),

    'lambda' : hp.loguniform('lambda', np.log(1e-6), np.log(10.0)),

}

In [14]:
max_evals = 20
trials = Trials()
history_opt = []
fmin(score,param_space,algo=tpe.suggest,trials=trials,max_evals=max_evals) #パラメータチューニングの実行

{'alpha': 1.0634009164333256e-07, 'colsample_bytree': 0.75, 'gamma': 0.05893544865964993, 'lambda': 0.1334350918496051, 'max_depth': 9, 'min_child_weight': 0.10443091626055466, 'subsample': 0.7000000000000001, 'booster': 'gbtree', 'objective': 'reg:squarederror', 'eta': 0.1, 'random_state': 71}
[0]	train-rmse:7.85888	eval-rmse:7.94578                                                                               

Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.                                   


Will train until eval-rmse hasn't improved in 100 rounds.                                                              

[100]	train-rmse:3.0308	eval-rmse:3.56303                                                                              

Stopping. Best iteration:                                                                                              
[60]	train-rmse:3.17276	eval-rmse:3.55757


[0]	train-rmse:7.85628	eval-rmse:7.93822               

{'alpha': 0.00011878133943498708,
 'colsample_bytree': 0.75,
 'gamma': 0.0002305197800505663,
 'lambda': 0.0002562104391838839,
 'max_depth': 5.0,
 'min_child_weight': 0.1344423118248131,
 'subsample': 0.7000000000000001}

In [15]:
#記録した情報からパラメータとスコアを取得
history_opt = sorted(history_opt,key=lambda tpl: tpl[1])
best = history_opt[0]
print(f"best params:{best[0]}, score:{best[1]:.4f}")

best params:{'alpha': 2.7452105248489394e-08, 'colsample_bytree': 0.9, 'gamma': 0.0002131414864855471, 'lambda': 0.002377748530473656, 'max_depth': 5, 'min_child_weight': 0.15566909647297286, 'subsample': 0.9, 'booster': 'gbtree', 'objective': 'reg:squarederror', 'eta': 0.1, 'random_state': 71}, score:3.5123
